In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import pickle
import os

In [2]:
def Return_retrieved_docs(retrieved_docs):
    if retrieved_docs:
        print(f" Retrieved {len(retrieved_docs)} documents!")
        for i, doc in enumerate(retrieved_docs):
            print(f"\n🔹 Chunk {i+1}:\n{doc.page_content}")
    else:
        print(" No documents retrieved!")

    return [doc.page_content for doc in retrieved_docs]


In [3]:
with open('Chunks/chunks_400_100.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

chunks=loaded_data

In [35]:
model_name="intfloat/e5-large-v2"
persist_dir = f"./chroma_db/{model_name.replace('/', '-')}"

model = HuggingFaceEmbeddings(model_name=model_name)

if os.path.exists(persist_dir):
    print("Loading existing vector database...")
    vector_store = Chroma(persist_directory=persist_dir, embedding_function=model)
else:
    print("Create a new vector database...")

doc_count = vector_store._collection.count()
print(f"Document count: {doc_count}")


Loading existing vector database...
Document count: 12520


In [36]:
query="What are the requirements for plumbing fixture clearances?"
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

In [37]:
retrieved_chunks=Return_retrieved_docs(retrieved_docs)
Returned_retrieved_chunks="\n".join(retrieved_chunks)
print(Returned_retrieved_chunks)

 Retrieved 5 documents!

🔹 Chunk 1:
vanity or closer than 30 inches (762 mm) center-to-cen-
ter between adjacent fixtures. There shall be a clearance
of not less than 21 inches (533 mm) in front of a water
closet, lavatory or bidet to any wall, fixture or door.
6. The location of piping, fixtures or equipment shall not
interfere with the operation of windows or doors.
7. In flood hazard areas as established by Table R301.2(1),

🔹 Chunk 2:
and other equipment and appliances to permit maintenance,
servicing and replacement. Clearances shall be maintained to
permit cleaning of heating and cooling surfaces; replacement
of filters, blowers, motors, controls and vent connections;
lubrication of moving parts; and adjustments.
Exception: Access shall not be required for ducts, piping,
or other components approved for concealment.

🔹 Chunk 3:
age piping only in the direction of drainage flow.
Exceptions:
1. Test tees serving as cleanouts.
2. A two-way cleanout installation that is approved
for 

In [38]:
import requests
import json

url = 'http://localhost:11434/api/generate'
payload = {
    "model": "llama3",
    "prompt": f"I will provide you text please do you use that text only to give answer for my query. Text:{Returned_retrieved_chunks}, query:{query}, "
}
data = json.dumps(payload)

response = requests.post(url, data=data, headers={'Content-Type': 'application/json'})

if response.status_code == 200:
    list_dict_words = []
    for each_word in response.text.split("\n"):
        try:
            data = json.loads(each_word) 
        except:
            pass
        list_dict_words.append(data)
        


llama_response = " ".join([word['response'] for word in list_dict_words if type(word) == type({})])
print(llama_response)

Based  on  the  provided  text ,  the  requirements  for  plumbing  fixture  clear ances  are :

 1 .  There  shall  be  a  clearance  of  not  less  than   21  inches  ( 533  mm )  in  front  of  a  water  closet ,  lav atory  or  bid et  to  any  wall ,  fixture  or  door .
 2 .  The  location  of  piping ,  fixtures  or  equipment  shall  not  interfere  with  the  operation  of  windows  or  doors .

 Additionally ,  there  is  a  minimum  clearance  of   12  inches  ( 305  mm )  between  the  fixture  and  the  nearest  point  of  a  storage  space .  
